<a href="https://colab.research.google.com/github/sqx27/AQS/blob/main/FM5252HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Alex Song FM5252 HW2


#S, the spot price of the asset at time t
#K, strike price of the option
#T, the maturity of the option. Time to maturity is defined as T−t
#N, the height of the binomial tree
#r, the risk-free interest rate, assumed to be constant between t and T
#q, the dividend yeild
#σ, volatility of underlying asset, the standard deviation of the asset returns

#calculate European Option prices using binomial tree
import numpy as np

S = 100
K = 100
T = 1
N = 3
r = 0.06
q = 0
sigma = 0.2
#check the difference between European and American option approach, put option is more obvious
option_type = 'put' 

def European_binomial_tree(S,K,T,N,r,q,sigma,option_type):
  t = T/N
  u = np.exp(sigma*np.sqrt(t))
  d = 1/u
  p = (np.exp((r-q)*t)-d)/(u-d)
  discount = np.exp(-r*t)

  #create a matrix with size (N+1)*(N+1) to place all the price values. (0,0) position is the original price
  ptree = np.zeros([N+1,N+1])
  #place all the values inside the matrix
  for i in range(N+1):
    for j in range(i+1):
      ptree[j,i] = S*(d**j)*(u**(i-j))
  #print(ptree)

  #create a second matrix to include all the value of options
  ovtree = np.zeros([N+1,N+1])
  #calculate the value of options at each point in the lattice
  if option_type == 'call':
    ovtree[:,N] = np.maximum(np.zeros(N+1), ptree[:,N]-K)
  else:
    ovtree[:,N] = np.maximum(np.zeros(N+1), K-ptree[:,N]) 

  #calculate the value back until we reach [0,0]
  for i in np.arange(N-1,-1,-1):
    for j in np.arange(0,i+1):
      ovtree[j,i] = discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1])
  #print(ovtree)
  delta = (ovtree[0,1]-ovtree[1,1])/(ptree[0,1]-ptree[1,1])
  gamma = ((ovtree[0,2]-ovtree[1,2])/(ptree[0,2]-ptree[1,2])-(ovtree[1,2]-ovtree[2,2])/(ptree[1,2]-ptree[2,2]))/(0.5*(ptree[0,2]-ptree[2,2]))

  return ovtree[0,0],delta,gamma;

option_price = European_binomial_tree(S,K,T,N,r,q,sigma,option_type)[0]
delta = European_binomial_tree(S,K,T,N,r,q,sigma,option_type)[1]
gamma = European_binomial_tree(S,K,T,N,r,q,sigma,option_type)[2]
dsigma = 0.0001
vega = (European_binomial_tree(S,K,T,N,r,q,sigma+dsigma,option_type)[0]-European_binomial_tree(S,K,T,N,r,q,sigma-dsigma,option_type)[0])/(2*dsigma)
dr = 0.0001
rho = (European_binomial_tree(S,K,T,N,r+dr,q,sigma,option_type)[0]-European_binomial_tree(S,K,T,N,r-dr,q,sigma,option_type)[0])/(2*dr)
dT = 0.0001
theta = -(European_binomial_tree(S,K,T+dT,N,r,q,sigma,option_type)[0]-European_binomial_tree(S,K,T-dT,N,r,q,sigma,option_type)[0])/(2*dT)


print('The price of an European ', option_type, ' option is ', option_price, '\nThe Delta value is ', delta, '\nThe Gamma value is ',gamma, 
      '\nThe Vega value is ',vega, '\nThe Rho value is ',rho, '\nThe Theta value is ',theta)

The price of an European  put  option is  5.728426535389136 
The Delta value is  -0.37194935661830325 
The Gamma value is  0.021176247761510354 
The Vega value is  40.493566000092684 
The Rho value is  -42.92336261666918 
The Theta value is  -1.473954907429409


In [ ]:
#calculate American Option prices using binomial tree
S = 100
K = 100
T = 1
N = 3
r = 0.06
q = 0
sigma = 0.2
#check the difference between European and American option approach, put option is more obvious
option_type = 'put'

def American_binomial_tree(S,K,T,N,r,q,sigma,option_type):
  t = T/N
  u = np.exp(sigma*np.sqrt(t))
  d = 1/u
  p = (np.exp((r-q)*t)-d)/(u-d)
  discount = np.exp(-r*t)

  #create a matrix with size (N+1)*(N+1) to place all the price values. (0,0) position is the original price
  ptree = np.zeros([N+1,N+1])
  #place all the values inside the matrix
  for i in range(N+1):
    for j in range(i+1):
      ptree[j,i] = S*(d**j)*(u**(i-j))
  #print(ptree)

  #create a second matrix to include all the value of options
  ovtree = np.zeros([N+1,N+1])
  #calculate the value of options at each point in the lattice, and calculate the value back until we reach [0,0]
  if option_type == 'call':
    ovtree[:,N] = np.maximum(np.zeros(N+1), ptree[:,N]-K)
    for i in np.arange(N-1,-1,-1):
      for j in np.arange(0,i+1):
        ovtree[j,i] = np.maximum(discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1]), ptree[j,i]-K)
  else:
    ovtree[:,N] = np.maximum(np.zeros(N+1), K-ptree[:,N]) 
    for i in np.arange(N-1,-1,-1):
      for j in np.arange(0,i+1):
        ovtree[j,i] = np.maximum(discount * (p*ovtree[j,i+1] + (1-p)*ovtree[j+1,i+1]), K-ptree[j,i])
  #print(ovtree)
  delta = (ovtree[0,1]-ovtree[1,1])/(ptree[0,1]-ptree[1,1])
  gamma = ((ovtree[0,2]-ovtree[1,2])/(ptree[0,2]-ptree[1,2])-(ovtree[1,2]-ovtree[2,2])/(ptree[1,2]-ptree[2,2]))/(0.5*(ptree[0,2]-ptree[2,2]))

  return ovtree[0,0],delta,gamma;

option_price = American_binomial_tree(S,K,T,N,r,q,sigma,option_type)[0]
delta = American_binomial_tree(S,K,T,N,r,q,sigma,option_type)[1]
gamma = American_binomial_tree(S,K,T,N,r,q,sigma,option_type)[2]
dsigma = 0.0001
vega = (American_binomial_tree(S,K,T,N,r,q,sigma+dsigma,option_type)[0]-American_binomial_tree(S,K,T,N,r,q,sigma-dsigma,option_type)[0])/(2*dsigma)
dr = 0.0001
rho = (American_binomial_tree(S,K,T,N,r+dr,q,sigma,option_type)[0]-American_binomial_tree(S,K,T,N,r-dr,q,sigma,option_type)[0])/(2*dr)
dT = 0.0001
theta = -(American_binomial_tree(S,K,T+dT,N,r,q,sigma,option_type)[0]-American_binomial_tree(S,K,T-dT,N,r,q,sigma,option_type)[0])/(2*dT)


print('The price of an American ', option_type, ' option is ', option_price, '\nThe Delta value is ', delta, '\nThe Gamma value is ',gamma, 
      '\nThe Vega value is ',vega, '\nThe Rho value is ',rho, '\nThe Theta value is ',theta)

The price of an American  put  option is  6.099357008892412 
The Delta value is  -0.4089772809232681 
The Gamma value is  0.02529746343125649 
The Vega value is  41.47172026729873 
The Rho value is  -39.518618048406395 
The Theta value is  -1.7760549375989143
